In [9]:
import pandas as pd


I part 2 starter vi ud med at indlæse vores 'train_set.csv', 'validation_set.csv' og 'test_set.csv'

In [10]:
train_data = pd.read_csv('train_set.csv')
val_data = pd.read_csv('validation_set.csv')
test_data = pd.read_csv('test_set.csv')

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score, f1_score


# Indlæs træningsdata

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'
    # Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier
    return 0  

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)

# Nu er 'y_train' klar uden risiko for NaN-værdier
X_train = train_data['stemmed_content']  # 'X_train' er også klar til brug

# Opret en pipeline med TF-IDF og logistisk regression, øg antallet af iterationer
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))  # Øg antallet af iterationer
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data['stemmed_content'].fillna('', inplace=True)
X_val = val_data['stemmed_content']
y_val = val_data['type'].apply(map_to_binary)

# Brug pipeline til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Forudsagte sandsynligheder (hvis du har brug for dem til MSE beregning)
y_pred_probs = pipeline.predict_proba(X_val)[:, 1]

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
# Bemærk: Dette er ikke standard for klassificering og anvendes normalt i regression.
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

Validation accuracy: 0.8769547738693467
LogisticRegression MSE: 0.08636638855705894


In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Beregn præcision, recall og F1-score
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Precision: 0.8751
Recall: 0.8195
F1-Score: 0.8464


Iforhold til meta data overvejede vi domain men i og med vi tidlgiere fandt at hver domain kun udgiver 1 type artikel mener vi at det vil "overfitte" vores model, hvor vores model bare genkender domain istedet for at forudsige. 